In [ ]:
from uproot_module import Data
from vbfdm import VBFDM
import uproot
import awkward
import pandas as pd
import numpy as np

In [ ]:
SIGNAL_PATH = "/home/santiago/VBF_DMSimp_spin0_EWKExcluded/Events/run_18/DMSimpSpin0_MY5000_MX1000_07042020.root"

vbf = VBFDM(SIGNAL_PATH, SIGNAL_PATH)

In [ ]:
vbf.cut_flow()